# Reconocimiento de Entidades Nombradas

El objetivo es identificar entidades nombradas en un corpus del area de genetica. Las etiquetas que se buscan indican si se trata de un gen o no. Debe tomarse en cuenta que las entidades nombradas pueden constar de mas de un elemento. Por tanto, se utiliza un etiquetado BIO.

Las etiquetas son como siguen:
1. __B-tag__ indica el inicio de una entidad (de izquierda a derecha).
2. __I-tag__ indica que la palabra pertenece a una entidad etiquetada con B-tag. En este sentido, siempre debe existir una etiqueta B-tag, pero no necesariamente una I-tag.
3. Finalmente la etiqueta __O__ indica que no se trata de una entidad nombrada.

## Cadenas Ocultas de Markov

Para dar solucion al problema propuesto se usara el modelo de aprendizaje automatico Hidden Markov Model (HMM). Es importante establecer que para construir un HMM se necesitan llevar a cabo tres etapas:

1. 
2. Decodificacion
3. Aprendizaje

### Construccion del modelo del lenguaje $\lambda$

Un modelo oculto de Markov se denota mediante la letra $\lambda$ y es una 5-tupla:

$$ \lambda = (S, O, A, B, \Pi) $$

Donde: 

* $S = \{s_1, ..., s_N\}$
* $O = \{o_1, ..., o_T\}$
* $A = \{a_{i,j}\} = p(q_{t+1}=S_j|q_t=S_i)$
* $B = \{b_{i,j}\} = p(q_t=o_i| q_t=Sj)$
* $\Pi = \{\Pi_i\} = p(q_1=S_i)$

In [144]:
import numpy as np
import re

## Lectura y preprocesamiento del corpus

El conjunto de estados ocultos $S$ se conformara por las etiquetas BIOS y el conjunto de simbolos de observacion $O$ seran las palabras.

In [275]:
with open('Final1/data_test.txt', 'r') as file:
    raw_corpus = file.read().splitlines()

In [276]:
raw_corpus

['IL-2\tB-DNA',
 'gene\tI-DNA',
 'expression\tO',
 'and\tO',
 'NF-kappa\tB-protein',
 'B\tI-protein',
 'activation\tO',
 'through\tO',
 'CD28\tB-protein',
 'requires\tO',
 'reactive\tO',
 'oxygen\tO',
 'production\tO',
 'by\tO',
 '5-lipoxygenase\tB-protein',
 '.\tO',
 '',
 'Activation\tO',
 'of\tO',
 'the\tO',
 'CD28\tB-protein',
 'surface\tI-protein',
 'receptor\tI-protein',
 'provides\tO',
 'a\tO',
 'major\tO',
 'costimulatory\tO',
 'signal\tO',
 'for\tO',
 'T\tO',
 'cell\tO',
 'activation\tO',
 'resulting\tO',
 'in\tO',
 'enhanced\tO',
 'production\tO',
 'of\tO',
 'interleukin-2\tB-protein',
 '(\tO',
 'IL-2\tB-protein',
 ')\tO',
 'and\tO',
 'cell\tO',
 'proliferation\tO',
 '.\tO',
 '',
 'In\tO',
 'primary\tB-cell_type',
 'T\tI-cell_type',
 'lymphocytes\tI-cell_type',
 'we\tO',
 'show\tO',
 'that\tO',
 'CD28\tB-protein',
 'ligation\tO',
 'leads\tO',
 'to\tO',
 'the\tO',
 'rapid\tO',
 'intracellular\tO',
 'formation\tO',
 'of\tO',
 'reactive\tO',
 'oxygen\tO',
 'intermediates\tO',
 '(

In [285]:
corpus = {
    'states': [],
    'obs': []
}

S = []
O = []


string_obs = ''
string_state = ''

for phrase in raw_corpus:
    if phrase == '' and len(string_obs) > 0 and len(string_state) > 0:
        corpus['states'].append(string_state)
        corpus['obs'].append(string_obs)
        string_obs = ''
        string_state = ''
    try:
        obs , state = phrase.split('\t')
        string_obs += obs + ' ' 
        string_state += state + ' '
        # Creando conjuntos de simbolos
        if obs not in O:
            O.append(obs)
        if state not in S:
            S.append(state)
    except:
        pass

In [287]:
corpus['states']

['B-DNA I-DNA O O B-protein I-protein O O B-protein O O O O O B-protein O ',
 'O O O B-protein I-protein I-protein O O O O O O O O O O O O O O B-protein O B-protein O O O O O ',
 'O B-cell_type I-cell_type I-cell_type O O O B-protein O O O O O O O O O O O O O O O O O O B-protein O O O O B-protein I-protein O B-protein I-protein O B-protein O O ',
 'O O O B-protein O O O O O O B-protein I-protein I-protein O O O O O O O B-protein I-protein O B-protein O ',
 'O O O O B-protein I-protein O O O O O O B-protein O O B-protein I-protein O O ',
 'O O O O O O O O O O O O O O O B-protein O O O ',
 'O B-DNA I-DNA I-DNA O B-DNA I-DNA I-DNA I-DNA I-DNA I-DNA O O B-cell_type O O O B-cell_type I-cell_type O ',
 'O O O O O O O O O O O O O O O O O O O O O O O O O ',
 'O O O O O O O O O O O O O O O O O O O ',
 'O O O O O O O O O O O B-DNA I-DNA O O O O O O O O O O ',
 'O O O O B-DNA I-DNA O O O O O O O O O O B-protein I-protein B-protein I-protein O O O B-DNA I-DNA I-DNA O O B-DNA I-DNA I-DNA I-DNA O O 

In [116]:
len_S = len(S)
len_O = len(O)

print('Numero de estados = ', len_S)
print('Alfabeto de observaciones = ', len_O)

Numero de estados =  11
Alfabeto de observaciones =  22053


In [117]:
freq_bigrams

{('IL-2', 'B-DNA'): 171,
 ('gene', 'I-DNA'): 1381,
 ('expression', 'O'): 3028,
 ('and', 'O'): 12075,
 ('NF-kappa', 'B-protein'): 1021,
 ('B', 'I-protein'): 1566,
 ('activation', 'O'): 2200,
 ('through', 'O'): 495,
 ('CD28', 'B-protein'): 128,
 ('requires', 'O'): 123,
 ('reactive', 'O'): 55,
 ('oxygen', 'O'): 50,
 ('production', 'O'): 466,
 ('by', 'O'): 4742,
 ('5-lipoxygenase', 'B-protein'): 7,
 ('.', 'O'): 18403,
 ('Activation', 'O'): 121,
 ('of', 'O'): 21102,
 ('the', 'O'): 19864,
 ('surface', 'I-protein'): 63,
 ('receptor', 'I-protein'): 857,
 ('provides', 'O'): 47,
 ('a', 'O'): 6534,
 ('major', 'O'): 146,
 ('costimulatory', 'O'): 26,
 ('signal', 'O'): 277,
 ('for', 'O'): 2826,
 ('T', 'O'): 406,
 ('cell', 'O'): 1465,
 ('resulting', 'O'): 83,
 ('in', 'O'): 12853,
 ('enhanced', 'O'): 219,
 ('interleukin-2', 'B-protein'): 74,
 ('(', 'O'): 5699,
 ('IL-2', 'B-protein'): 562,
 (')', 'O'): 5745,
 ('proliferation', 'O'): 356,
 ('In', 'O'): 1415,
 ('primary', 'B-cell_type'): 108,
 ('T', 'I-c

In [122]:
freq_states

{'B-DNA': 9532,
 'I-DNA': 15773,
 'O': 382758,
 'B-protein': 30239,
 'I-protein': 24826,
 'B-cell_type': 6711,
 'I-cell_type': 8734,
 'B-cell_line': 3828,
 'I-cell_line': 7379,
 'B-RNA': 951,
 'I-RNA': 1530}

## Creacion del modelo del lenguaje

El modelo del lenguaje sera construido al hacer las matrices $A$, $B$ y $\Pi$, usando los alfabetos $S$ y $O$ obtenidos previamente.

In [102]:
# Inicializacion de matrices
A = np.zeros((len_S, len_S))
B = np.zeros((len_S, len_O))
Pi = np.zeros(len_S)

print(A.shape)
print(B.shape)
print(Pi.shape)

(11, 11)
(11, 22053)
(11,)


In [ ]:
def smoothingLaplacian(wi, wj, N, sigma):
    if sigma == 'states':
        return (freq_bigrams[(wi, wj)] + 1) / (freq_states[wj] + N)
    elif sigma == 'symbols':
        return (freq_bigrams[(wi, wj)] + 1) / (freq_states[wj] + N)